In [21]:
import random

numbers_list = []
last = 0
for i in range(10_000_000):
    last += random.randint(1, 10)
    numbers_list += [last]

numbers_list[0:5]

[9, 13, 16, 23, 30]

In [22]:
import math


def btree_organize(numbers_list):
    if len(numbers_list) < 10:
        return numbers_list 

    middle = numbers_list[math.floor(len(numbers_list) / 2) - 1]
    smaller = [x for x in numbers_list if x < middle]
    bigger = [x for x in numbers_list if x > middle]
    return { "middle": middle, "left": btree_organize(smaller), "right": btree_organize(bigger) }


numbers_btree = btree_organize(numbers_list)
str(numbers_btree)[0:400] + "..."

"{'middle': 27503539, 'left': {'middle': 13753203, 'left': {'middle': 6872625, 'left': {'middle': 3438531, 'left': {'middle': 1718337, 'left': {'middle': 859411, 'left': {'middle': 429492, 'left': {'middle': 214919, 'left': {'middle': 107474, 'left': {'middle': 53714, 'left': {'middle': 26759, 'left': {'middle': 13434, 'left': {'middle': 6780, 'left': {'middle': 3351, 'left': {'middle': 1728, 'left..."

In [26]:
def btree_search(number, numbers_btree, steps=0):
    if 'middle' in numbers_btree:
        if number < numbers_btree['middle']:
            return btree_search(number, numbers_btree['left'], steps + 1)
        elif number > numbers_btree['middle']:
            return btree_search(number, numbers_btree['right'], steps + 1)
        else:
            return { "steps": steps, "found": numbers_btree['middle'] }
    else:
        if number in numbers_btree:
            steps += numbers_btree.index(number) + 1
            return { "steps": steps, "found": number }
        else:
            return { "steps": steps, "found": None }
        
btree_search(424242, numbers_btree)

{'steps': 25, 'found': 424242}

In [28]:
from statistics import mean

sample = random.sample(numbers_list, 1000)

%time steps = [btree_search(s, numbers_btree)['steps'] for s in sample]
print("Avg number of btree steps", mean(steps))
print("\n")
%time indexes = [numbers_list.index(s) for s in sample]
print("Avg number of indexes", mean(indexes))

CPU times: user 31.7 ms, sys: 1.4 ms, total: 33.1 ms
Wall time: 33.5 ms
Avg number of btree steps 23.794


CPU times: user 3min 27s, sys: 1.98 s, total: 3min 28s
Wall time: 3min 40s
Avg number of indexes 4952640.658


In [29]:
import numpy as np
from sklearn.linear_model import LinearRegression

X = np.array(numbers_list).reshape(-1, 1)
y = np.array(range(0, len(numbers_list)))

model = LinearRegression().fit(X, y)
model.score(X, y)

0.9999999797989778

In [38]:
def predict_index(number, model):
    return math.ceil(model.predict([[number]])[0])


print("Predicted index", predict_index(19123334, model), "Actual index", numbers_list.index(19123334))

Predicted index 3476815 Actual index 3476656


In [39]:
def regression_search(number, numbers_list, model, steps=0):
    maximum = len(numbers_list) - 1
    predicted_index = min(predict_index(number, model), maximum)
    if numbers_list[predicted_index] == number:
        return { "steps": steps, "found": number }
    
    move = 1
    while True:
        steps += 1
        next_number = numbers_list[predicted_index + move]
        if number > next_number:
            move += 1
        elif number < next_number:
            move -= 1
        else:
            return { "steps": steps + 1, "found": number }
        
    
regression_search(19123334, numbers_list, model)

{'steps': 162, 'found': 19123334}

In [40]:
sample = random.sample(numbers_list, 1000)

%time steps = [btree_search(s, numbers_btree)['steps'] for s in sample]
print("Avg number of btree steps", mean(steps))
print("\n")

%time steps = [regression_search(s, numbers_list, model)['steps'] for s in sample]
print("Avg number of regression steps", mean(steps))
print("\n")

%time indexes = [numbers_list.index(s) for s in sample]
print("Avg number of indexes", mean(indexes))

CPU times: user 34.1 ms, sys: 4.52 ms, total: 38.7 ms
Wall time: 43.6 ms
Avg number of btree steps 23.548


CPU times: user 303 ms, sys: 13.7 ms, total: 316 ms
Wall time: 305 ms
Avg number of regression steps 350.998


CPU times: user 3min 24s, sys: 1.62 s, total: 3min 26s
Wall time: 3min 31s
Avg number of indexes 5052603.604


In [41]:
%%bash

pip install pympler hurry.filesize

In [42]:
from pympler import asizeof
from hurry.filesize import size

list_size = asizeof.asizeof(numbers_list)
print("Size of list", size(list_size))
print("Size of btree", size(asizeof.asizeof(numbers_btree)))
print("Size of model", size(asizeof.asizeof(model)))

Size of list 382M
Size of btree 754M
Size of model 1K


# Conclusion

The regression model being used to find an item in the list is still not faster then the b-tree, although it can jump to a position very close to the desired index, it has to walk left or right a little bit until it finds the item

On the other hand, the regression model is not too slow either, 300ms might be acceptable for 10 million items, and it's way lighter (<1K), although you do need the original list weight anyway to find the item (382M), you don't pay the price for the whole b-tree structure (754M)